In [ ]:
# 一応動いたけど学習率が上がらない！
# エポック数;100, %
import os
import glob
import csv
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

# ファイル単位でのデータ分割
fix_F_A_dir = "fix_WF_A"
csv_files = glob.glob(os.path.join(fix_F_A_dir, "*.csv"))

# ランダムシャッフル
random.seed(42)  # 再現性のためにシードを固定
random.shuffle(csv_files)

# 訓練データ: 80%、テストデータ: 20%に分割
split_ratio = 0.8
split_index = int(len(csv_files) * split_ratio)
train_files = csv_files[:split_index]
test_files = csv_files[split_index:]

print(f"Train files: {len(train_files)}, Test files: {len(test_files)}")

class AccentDataset(Dataset):
    def __init__(self, files):
        self.data = []
        self.missing_accent_count = 0  # 空白や指定されていないラベルのカウント用

        for csv_file in files:
            with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    # 特徴量（Feature_0〜Feature_767）を抽出
                    feature = [float(row[f"feature_{i}"]) for i in range(768)]

                    # ラベル（Accent）の確認
                    accent_value = row.get("Accent", "").strip()  # 空白や未指定の確認用
                    if not accent_value:  # 空白または指定なしの場合
                        self.missing_accent_count += 1
                        continue  # スキップして次の行へ

                    try:
                        # ラベルを数値変換
                        label = int(float(accent_value))
                        self.data.append((feature, label))
                    except ValueError:
                        # 万が一ラベルが不正な形式の場合もカウント
                        self.missing_accent_count += 1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features, label = self.data[idx]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# データセットの作成
train_dataset = AccentDataset(train_files)
test_dataset = AccentDataset(test_files)

# DataLoaderの作成
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# モデル定義
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformerModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, 512)  # 入力層 → 隠れ層1
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(512, 256)       # 隠れ層1 → 隠れ層2
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(256, 128)       # 隠れ層2 → 隠れ層3
        self.relu3 = nn.ReLU()
        self.linear4 = nn.Linear(128, output_dim)  # 隠れ層3 → 出力層

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        return x

# ハイパーパラメータ
input_dim = 768  # 特徴量次元
output_dim = 2   # クラス数（高: 1, 低: 0）
lr = 0.001
epochs = 100

# モデル、損失関数、オプティマイザの準備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel(input_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# 損失と精度を保存するリスト
train_losses = []
test_accuracies = []

# 訓練ループ
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        # フォワード
        outputs = model(features)
        loss = criterion(outputs, labels)

        # バックプロパゲーションと最適化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # 平均損失を保存
    train_losses.append(total_loss / len(train_loader))
    print(f"Epoch {epoch+1}/{epochs}, Loss: {train_losses[-1]:.4f}")

    # テストデータでの評価
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # 精度を保存
        accuracy = 100 * correct / total
        test_accuracies.append(accuracy)
        print(f"Test Accuracy: {accuracy:.2f}%")

# 学習結果をmatplotlibで描画
plt.figure(figsize=(12, 5))

# 損失のプロット
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses, marker='o', label='Train Loss')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.legend()

# 精度のプロット
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), test_accuracies, marker='o', label='Test Accuracy')
plt.title('Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.grid()
plt.legend()

# 結果を保存および表示
plt.tight_layout()
plt.savefig("training_results_WF_A_100epo_bach16_hid3.png")
plt.show()

# テスト結果をファイルに保存
predictions_dir = "predictions_WF_A_100epochs_bach16_hid3"
os.makedirs(predictions_dir, exist_ok=True)

model.eval()
with torch.no_grad():
    for csv_file in test_files:
        file_name = os.path.basename(csv_file)
        output_file = os.path.join(predictions_dir, f"pred_{file_name}")

        # テストファイルの読み込み
        features = []
        labels = []
        with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                feature = [float(row[f"feature_{i}"]) for i in range(768)]
                features.append(feature)

        # テンソルに変換
        features = torch.tensor(features, dtype=torch.float32).to(device)

        # モデルによる予測
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)

        # Mora, Accent形式で保存
        results = [(f"Mora_{i + 1}", int(predicted[i].cpu().item())) for i in range(features.size(0))]
        with open(output_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["Mora", "Accent"])  # ヘッダー行
            writer.writerows(results)

        print(f"Predictions for {file_name} saved to: {output_file}")